In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import urllib.parse

car_info = []
car_model = []
car_brand = []
car_price = []
car_pic = []
car_km = []
fuel_type = []
transmission = []
year = []

In [2]:
for page_number in range(1, 20):
    url_page_numbers = f"https://www.standvirtual.com/carros?page={page_number}"
    response_page_numbers = requests.get(url_page_numbers)
    soup_page_numbers = BeautifulSoup (response_page_numbers.content)

    auto_components = soup_page_numbers.find_all("main", class_="ooa-19sk4h4 era12cr0")
    for component in auto_components:
        car_standvirtual1 = component.find_all("div", class_="ooa-8mr5l1") #ok
        for component in car_standvirtual1:
            car_standvirtual2 = component.find_all("div", class_="ooa-ys55sm er8sc6m1") #ok
            for component in car_standvirtual2:
                car_standvirtual3 = component.find_all("div", class_="ooa-1un3d5b er8sc6m2") #ok
                for component in car_standvirtual3:
                    car_standvirtual4 = component.find_all("div", class_="ooa-1hab6wx er8sc6m9") #ok
                    for component in car_standvirtual4:
                        car_standvirtual5 = component.find_all("article", class_="ooa-1t80gpj ev7e6t818") #ok
                        for component in car_standvirtual5:
                            car_standvirtual6 = component.find_all("section", class_="ooa-1rnjex3 ev7e6t817") #ok

                            #car brand and model (car_info)
                            for component in car_standvirtual6:
                                car_standvirtual_name = component.find_all("div", class_="ooa-1jgmfmo ev7e6t812") #ok
                                #if any("citroën" in name.get_text().lower() for name in car_standvirtual_name):
                                title_ = component.find('h1')
                                car_info.append(title_.text.strip())

                            #price (car_price)
                            for component in car_standvirtual6:
                                car_standvirtual_price = component.find_all("div", class_="ooa-j0d5pq ev7e6t813") #ok
                                for component in car_standvirtual_price:
                                    car_standvirtual_price2 = component.find_all("div", class_="ooa-1t7cj0c ermhljg4") #ok
                                    for component in car_standvirtual_price2:
                                        car_standvirtual_price3 = component.find_all("div", class_="ooa-1wb7q8u ev7e6t814") #ok
                                        for component in car_standvirtual_price3:
                                            price = component.h3.get_text(strip=True)
                                            car_price.append(price)

                            for component in car_standvirtual6:
                                # Check for the 'dd' elements
                                car_info_km = car_info_fuel = car_info_transmission = car_info_year = None

                                _info_km = component.find('dd', {'class': 'ooa-1smopdt ev7e6t85', 'data-parameter': 'mileage'})
                                if _info_km:
                                    car_info_km = _info_km.get_text(strip=True)
                                    car_km.append(car_info_km)
                                else:
                                    car_km.append("None")

                                _info_fuel = component.find('dd', {'class': 'ooa-1smopdt ev7e6t85', 'data-parameter': 'fuel_type'})
                                if _info_fuel:
                                    car_info_fuel = _info_fuel.get_text(strip=True)
                                    fuel_type.append(car_info_fuel)
                                else:
                                    fuel_type.append("None")

                                _info_transmission = component.find('dd', {'class': 'ooa-1smopdt ev7e6t85', 'data-parameter': 'gearbox'})
                                if _info_transmission:
                                    car_info_transmission = _info_transmission.get_text(strip=True)
                                    transmission.append(car_info_transmission)
                                else:
                                    transmission.append("None")  # Append None for missing data

                                _info_year = component.find('dd', {'class': 'ooa-1smopdt ev7e6t85', 'data-parameter': 'first_registration_year'})
                                if _info_year:
                                    car_info_year = _info_year.get_text(strip=True)
                                    year.append(car_info_year)
                                else:
                                    year.append("None")  # Append None for missing data

                                # Extract the image URL (with a check)
                                car_standvirtual_img = component.find("div", class_="ooa-165sslu e1j52kpv10")
                                if car_standvirtual_img:
                                    pic_ = car_standvirtual_img.find('img')
                                    if pic_:
                                        car_pic.append(pic_['src'])
                                    else:
                                        car_pic.append("None")
                                else:
                                    car_pic.append("None")

DATA FRAME BUILDUP

In [3]:
car_data = {
    'car_full_name': car_info,
    "registration_year": year,
    "km_to_date": car_km,
    "sale_price": car_price,
    "fuel_type": fuel_type,
    "transmission_type": transmission,
    "sale_picture_link": car_pic
}

In [4]:
standvirtual_df = pd.DataFrame(car_data)

# Clear the DataFrame

standvirtual_df['car_full_name'] = standvirtual_df['car_full_name'].str.replace('ë', 'e')
standvirtual_df['car_full_name'] = standvirtual_df['car_full_name'].str.replace('é', 'e')
standvirtual_df['car_full_name'] = standvirtual_df['car_full_name'].str.lower()

standvirtual_df = standvirtual_df[standvirtual_df['registration_year'] != "None"]
standvirtual_df['registration_year'] = standvirtual_df['registration_year'].str.replace(' ', '').astype(int)

standvirtual_df = standvirtual_df[standvirtual_df['km_to_date'] != "None"]
standvirtual_df['km_to_date'] = standvirtual_df['km_to_date'].str.replace('km', '').str.replace(' ', '').astype(int)

standvirtual_df = standvirtual_df[standvirtual_df['sale_price'] != "None"]
standvirtual_df['sale_price'] = standvirtual_df['sale_price'].str.replace(' ', '')
standvirtual_df['sale_price'] = standvirtual_df['sale_price'].astype(int)

standvirtual_df = standvirtual_df[standvirtual_df['fuel_type'] != "None"]
standvirtual_df['fuel_type'] = standvirtual_df['fuel_type'].str.replace('é', 'e')
standvirtual_df['fuel_type'] = standvirtual_df['fuel_type'].str.lower()
standvirtual_df['fuel_type'] = standvirtual_df['fuel_type'].str.replace(r'híbrido \(gasolina\)', 'hibrido', regex=True)

standvirtual_df = standvirtual_df[standvirtual_df['transmission_type'] != "None"]
standvirtual_df['transmission_type'] = standvirtual_df['transmission_type'].str.replace('á', 'a')
standvirtual_df['transmission_type'] = standvirtual_df['transmission_type'].str.lower()


In [5]:
car_brands = [
    "toyota", "ford", "chevrolet", "honda", "volkswagen", "nissan",
    "hyundai", "bmw", "mercedes-benz", "audi", "subaru", "kia",
    "mazda", "tesla", "jeep", "gmc", "volvo", "jaguar", "land rover",
    "porsche", "ferrari", "lamborghini", "aston martin", "rolls-royce",
    "bentley", "maserati", "alfa romeo", "mini", "fiat", "peugeot",
    "renault", "citroen", "skoda", "mitsubishi", "infiniti", "acura",
    "lexus", "buick", "chrysler", "dodge", "ram", "cadillac", "lincoln",
    "genesis", "suzuki", "vw", "ds", "seat", "opel", "dacia"
]

In [6]:
# Create a new column 'car_brand' with the brand name if it exists in 'brand_model'
standvirtual_df['car_brand'] = standvirtual_df['car_full_name'].str.extract(f'({"|".join(car_brands)})', expand=False)

# Fill any NaN values in the 'car_brand' column with an empty string
standvirtual_df['car_brand'] = standvirtual_df['car_brand'].fillna('')

cols = ['car_brand'] + [col for col in standvirtual_df if col != 'car_brand']
standvirtual_df = standvirtual_df[cols]

In [7]:
standvirtual_df.to_csv('standvirtual_df.csv', index=False)
